In [3]:
import tensorflow
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import GlobalMaxPooling2D
import firebase_admin
from firebase_admin import db, credentials

In [4]:
cred=credentials.Certificate("firebase connection key")
# firebase_Admin.initalize_app(cred,{"databaseURL":})
firebase_admin.initialize_app(cred, {
    'storageBucket': 'storage buckett',
})

In [5]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

In [6]:
model=tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Glob  (None, 2048)              0         
 alMaxPooling2D)                                                 
                                                                 
Total params: 23587712 (89.98 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [7]:
import cv2
import numpy as np
from numpy.linalg import norm

In [10]:
from firebase_admin import storage
import io
import cv2
import numpy as np

# Initialize Firebase and get the storage bucket
# Make sure to use just the bucket name, not the full URL
bucket = storage.bucket("dressmedaily-e8e0c.appspot.com")

# Create a reference to the image file in the "Dataset" directory
# Correctly specify the path to the image within the bucket
image_blob = bucket.blob("Dataset/2837.jpg")

# Download the image to a BytesIO object
image_data = io.BytesIO()
image_blob.download_to_file(image_data)
image_data.seek(0) # Move the file pointer to the beginning of the file

# Read the image data into a NumPy array
image_array = np.frombuffer(image_data.read(), dtype=np.uint8)
img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

# Resize the image to 224x224 pixels
img_resized = cv2.resize(img, (224, 224))

# Convert the resized image to a suitable format (JPEG)
is_success, buffer = cv2.imencode(".jpg", img_resized)
io_buf = io.BytesIO(buffer)

# Create a reference to the "resizedimg" directory and the image file
resized_image_blob = bucket.blob("resizedimg/2837_resized.jpg")

# Upload the resized image to Firebase Storage
resized_image_blob.upload_from_file(io_buf, content_type='image/jpeg')


In [6]:
img=np.array(img)

In [7]:
img.shape

(224, 224, 3)

In [8]:
expand_img=np.expand_dims(img, axis=0)

In [9]:
expand_img.shape

(1, 224, 224, 3)

In [10]:
pre_img=preprocess_input(expand_img)

In [11]:
pre_img

array([[[[151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         ...,
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ]],

        [[151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         ...,
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ]],

        [[151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         ...,
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ]],

        ...,

        [[151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         [151.061  , 138.22101, 131.32   ],
         ...,


In [12]:
pre_img.shape

(1, 224, 224, 3)

In [13]:
result=model.predict(pre_img).flatten()

1/1 [==============================] - 1s 719ms/step


In [14]:
result.shape

(2048,)

In [15]:
normalized=result/norm(result)

In [16]:
normalized.shape

(2048,)

In [17]:
def extract_feature(img_path, model):
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    img=np.array(img)
    expand_img=np.expand_dims(img, axis=0)
    pre_img=preprocess_input(expand_img)
    result=model.predict(pre_img).flatten()
    normalized=result/norm(result)
    return normalized

In [18]:
extract_feature("2837.jpg",model)

1/1 [==============================] - 0s 75ms/step


array([0.00596654, 0.00037371, 0.00811542, ..., 0.01471429, 0.01509576,
       0.02696898], dtype=float32)

In [19]:
import os
from tqdm import tqdm

In [20]:
filename=[]
feature_list=[]

for file in os.listdir('Dataset'):
    filename.append(os.path.join('Dataset',file))

In [21]:
filename[0:5]

['Dataset\\1529.jpg',
 'Dataset\\1531.jpg',
 'Dataset\\1532.jpg',
 'Dataset\\1533.jpg',
 'Dataset\\1534.jpg']

In [22]:
for file in tqdm(filename):
    feature_list.append(extract_feature(file,model))


  0%|                                                                                          | 0/972 [00:00<?, ?it/s]

1/1 [==============================] - 0s 62ms/step



  0%|                                                                                  | 1/972 [00:00<02:16,  7.09it/s]

1/1 [==============================] - 0s 85ms/step



  0%|▏                                                                                 | 2/972 [00:00<02:06,  7.64it/s]

...


100%|███████████████████████████████████████████████████████████████████████████████▊| 970/972 [03:39<00:00,  4.23it/s]

1/1 [==============================] - 0s 94ms/step



100%|███████████████████████████████████████████████████████████████████████████████▉| 971/972 [03:40<00:00,  4.35it/s]

1/1 [==============================] - 0s 94ms/step


100%|████████████████████████████████████████████████████████████████████████████████| 972/972 [03:40<00:00,  4.41it/s]


In [23]:
import pickle
pickle.dump(feature_list,open('featurevector.pkl','wb'))
pickle.dump(filename,open('filename.pkl','wb'))

In [11]:
from firebase_admin import storage
import io
import cv2
import numpy as np
import os
from tqdm import tqdm
import pickle

# Initialize Firebase and get the storage bucket
bucket = storage.bucket("dressmedaily-e8e0c.appspot.com")

def download_image_from_firebase(image_path):
    # Create a reference to the image file in the "Dataset" directory
    image_blob = bucket.blob(image_path)

    # Download the image to a BytesIO object
    image_data = io.BytesIO()
    image_blob.download_to_file(image_data)
    image_data.seek(0) # Move the file pointer to the beginning of the file

    # Read the image data into a NumPy array
    image_array = np.frombuffer(image_data.read(), dtype=np.uint8)
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    return img

def extract_feature(image_path, model):
    img = download_image_from_firebase(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.array(img)
    expand_img = np.expand_dims(img, axis=0)
    pre_img = preprocess_input(expand_img)
    result = model.predict(pre_img).flatten()
    normalized = result / np.linalg.norm(result)
    return normalized

# Assuming 'model' is already defined and initialized elsewhere in your code

filename = []
feature_list = []

# List all files in the "Dataset" directory
blobs = bucket.list_blobs(prefix="Dataset/")

for blob in tqdm(blobs):
    # Exclude directories
    if not blob.name.endswith('/'):
        filename.append(blob.name)
        feature_list.append(extract_feature(blob.name, model))

pickle.dump(feature_list, open('featurevector.pkl', 'wb'))
pickle.dump(filename, open('filename.pkl', 'wb'))



1it [00:01,  1.41s/it]

1/1 [==============================] - 1s 856ms/step



2it [00:03,  1.72s/it]

1/1 [==============================] - 0s 74ms/step



3it [00:04,  1.59s/it]

...


499it [07:54,  1.19it/s]

1/1 [==============================] - 0s 66ms/step



500it [07:55,  1.19it/s]

1/1 [==============================] - 0s 65ms/step


501it [07:55,  1.05it/s]


In [12]:
filename[0:5]

['Dataset/2837.jpg',
 'Dataset/2838.jpg',
 'Dataset/2840.jpg',
 'Dataset/2841.jpg',
 'Dataset/2842.jpg']